In [1]:
%load_ext autoreload
%autoreload 2
# Set-up: Import numpy and assign GPU


import os
os.environ['TRANSFORMERS_CACHE'] = '/local/helenl/.cache/'
os.environ['PYTORCH_TRANSFORMERS_CACHE'] = '/local/helenl/.cache/'

from collections import defaultdict


import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from tqdm.notebook import tqdm

from gpu_utils import restrict_GPU_pytorch
restrict_GPU_pytorch('3')

Using GPU:3


In [2]:
# Load Amazon WILDS pre-trained model

import statistics
import sys
import pickle
import numpy as np
import pandas as pd
import statistics


import torch
import torchvision.transforms as transforms

import argparse
import pdb

from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader, get_eval_loader
from wilds.common.grouper import CombinatorialGrouper
from transforms_helenl import initialize_transform, getBertTokenizer
from wordcloud import WordCloud

sys.path.insert(0, './wilds/examples/')
os.environ["MODEL_DIR"] = '/local/helenl/helenl/models/'
model_dir = '/local/helenl/helenl/models'

from algorithms.initializer import initialize_algorithm


In [3]:
import random

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
set_seed(0)

In [4]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
#config_dict = pickle.load(open('amazon_config.txt', 'rb'))



full_dataset = get_dataset(dataset='civilcomments', download=False, root_dir = './wilds/data')

In [ ]:
infile = open('./model_configs/civilcomments_ERM_config.txt','rb')
new_dict = vars(pickle.load(infile))
infile.close() 

print(new_dict)

In [ ]:
def predict_augmented_labels(transform_name
                            , num_samples = 4
                            , aug_char_min = 1
                            , aug_char_max = 1
                            , aug_char_p = 1
                            , aug_word_min = 1
                            , aug_word_max = None
                            , aug_word_p = 0.1
                            , min_char = 4):

    # Grab config generated by WILDS library from txt file
    infile = open('./model_configs/civilcomments_GroupDRO_config.txt','rb')
    new_dict = vars(pickle.load(infile))
    infile.close() 

    # Modify config with intended transformation
    new_dict['transform'] = transform_name

    # Create config with ERM algorithm
    config = Namespace(**new_dict)

    
    # Generate training and evaluation transforms
    print("num_samples:", num_samples)
    train_transform = initialize_transform(transform_name=config.transform
                                          , config=config
                                          , dataset=full_dataset
                                          , is_training=True
                                          , num_samples = num_samples
                                          , aug_char_min = aug_char_min
                                          , aug_char_max = aug_char_max
                                          , aug_char_p = aug_char_p
                                          , aug_word_min = aug_word_min
                                          , aug_word_max = aug_word_max
                                          , aug_word_p = aug_word_p
                                          , min_char = min_char)
    
    # Prepare training data, loader, and grouper
    train_data = full_dataset.get_subset('train', transform=train_transform)
    train_loader = get_train_loader('standard', train_data, batch_size=64)
    train_grouper = CombinatorialGrouper(dataset=full_dataset, groupby_fields=config.groupby_fields)
    
    # Prepare training data, loader, and grouper
    eval_transform = initialize_transform(transform_name=config.transform
                                          , config=config
                                          , dataset=full_dataset
                                          , is_training=False
                                          , num_samples = num_samples
                                          , aug_char_min = aug_char_min
                                          , aug_char_max = aug_char_max
                                          , aug_char_p = aug_char_p
                                          , aug_word_min = aug_word_min
                                          , aug_word_max = aug_word_max
                                          , aug_word_p = aug_word_p
                                          , min_char = min_char
                                        )
    
    
    eval_data = full_dataset.get_subset('test', transform=eval_transform)
    #return (eval_data.indices, eval_data.dataset)    


    eval_loader = get_eval_loader('standard', eval_data, batch_size = 64)
    
    # CODE TAKEN FROM WILDS TRAINING SCRIPTS:
    datasets = defaultdict(dict)
    for split in full_dataset.split_dict.keys():
        if split=='train':
            transform = train_transform
            verbose = True
        elif split == 'val':
            transform = eval_transform
            verbose = True
        else:
            transform = eval_transform
            verbose = False
        # Get subset
        datasets[split]['dataset'] = full_dataset.get_subset(
            split,
            frac=config.frac,
            transform=transform)

        if split == 'train':
            datasets[split]['loader'] = get_train_loader(
                loader=config.train_loader,
                dataset=datasets[split]['dataset'],
                batch_size=config.batch_size,
                uniform_over_groups=config.uniform_over_groups,
                grouper=train_grouper,
                distinct_groups=config.distinct_groups,
                n_groups_per_batch=config.n_groups_per_batch,
            **config.loader_kwargs)
        else:
            datasets[split]['loader'] = get_eval_loader(
                loader=config.eval_loader,
                dataset=datasets[split]['dataset'],
                grouper=train_grouper,
                batch_size=config.batch_size,
                **config.loader_kwargs)

    # Set fields
    datasets[split]['split'] = split
    datasets[split]['name'] = full_dataset.split_names[split]
    datasets[split]['verbose'] = verbose

    # Loggers
    # datasets[split]['eval_logger'] = BatchLogger(
    #     os.path.join(config.log_dir, f'{split}_eval.csv'), mode=mode, use_wandb=(config.use_wandb and verbose))
    # datasets[split]['algo_logger'] = BatchLogger(
    #     os.path.join(config.log_dir, f'{split}_algo.csv'), mode=mode, use_wandb=(config.use_wandb and verbose))

    print(transform_name)
    print("initialize model")
    # Initiate model and run on training set
    alg = initialize_algorithm(config, datasets, train_grouper)
    
    # TODO: pytorch load pretrained weights
    #alg.model.load_state_dict(torch.load('./best_model.pth'))
    alg.load_state_dict(torch.load('./best_models/civilcomments_GroupDRO_best_model.pth')['algorithm'])
    alg.model.cuda()
    
    print("initialization complete")
    
    
    print("generating predictions")
    it = iter(eval_loader)
    predictions = []
    labels = []
    metadata = []
    
    
    for batch in tqdm(it):

        for sample in batch[0]:
            raw_pred = alg.model(sample.cuda()).cpu().detach().numpy()
            
            predictions.extend(raw_pred.tolist())
        
        for i in range(num_samples):
            labels.extend(batch[1].tolist())
            metadata.extend(batch[2].tolist()) # tensor of (64,16) for each batch


    
    print("writing predictions")
    file_name = './data/civilcomments_DRO_predictions_optimized_params/' + transform_name + ".npy"
    #file_name = './ERM_word_frequency_experiment/' + transform_name + "_aug_size_" + str(aug_size) + ".npy"
    #file_name = './ERM_word_num_samples_experiment/' + transform_name + "_num_samples_" + str(num_samples) + ".npy"
    #file_name = './ERM_char_frequency_experiment_2/word_p_' + str(aug_word_p) + transform_name + "_aug_size_" + str(aug_char_min) + ".npy"

    #parameters = locals()                      
    with open(file_name, 'wb+') as file:
        np.save(file, predictions)
        np.save(file, labels)
        np.save(file, metadata)
        #np.save(file, parameters)


    return (eval_data.indices, eval_data.dataset)    


In [6]:
char_augment_labels = [#'bert'
                           #, 'nlp_ocr'
                           #, 'nlp_keyboard'
                        #'nlp_random_char_insert'
                           'nlp_random_char_substitution'
                           , 'nlp_random_char_swap'
                           , 'nlp_random_char_deletion'
                      ]

word_augment_labels = ['nlp_spelling_substitution'
                           , 'nlp_random_word_swap'
                           , 'nlp_random_word_delete'
                           , 'nlp_random_word_substitute'
                           , 'nlp_antonym'
                           , 'nlp_wordnet_synonym'
                           , 'nlp_ppdb_synonym'
                           , 'nlp_random_contextual_word_insertion_bert_cased_embedding'
                           , 'nlp_random_contextual_word_insertion_distilbert_uncased_embedding'
                           , 'nlp_random_contextual_word_insertion_distilbert_cased_embedding'
                           , 'nlp_random_contextual_word_insertion_roberta_base_embedding'
                           , 'nlp_random_contextual_word_insertion_distilroberta_base_embedding',
                           ]

in_progress = ['nlp_random_crop'
               , 'nlp_back_translation_aug'
                           , 'nlp_contextual_sentence_insertion_gpt2_embedding'
                           , 'nlp_contextual_sentence_insertion_xlnet_cased_embedding'
                           , 'nlp_contextual_sentence_insertion_distilgpt2_embedding'
                           , 'nlp_abstractive_summarization_bart_large_cnn'
                           , 'nlp_abstractive_summarization_t5_small'
                           , 'nlp_abstractive_summarization_t5_base'
                           , 'nlp_abstractive_summarization_t5_large'
                           , 'nlp_back_translation_aug'
               , 'nlp_random_similar_word_insertion_word2vec_embedding'
                           , 'nlp_random_similar_word_insertion_glove_embedding'
                           , 'nlp_random_similar_word_insertion_fasttext_embedding'
                           , 'nlp_random_similar_word_substitution_tfidf_embedding'
                           , 'nlp_random_similar_word_insertion_tfidf_embedding'
                           , 'nlp_random_similar_word_substitution_word2vec_embedding'
                           , 'nlp_random_similar_word_substitution_glove_embedding'
                           , 'nlp_random_similar_word_substitution_fasttext_embedding'
              ]


word_embedding_based_augmentations = [
                           #, 'nlp_random_contextual_word_insertion_xlnet_embedding'
                           'nlp_random_contextual_word_insertion_bart_base_embedding'
                           , 'nlp_random_contextual_word_insertion_squeezebert_uncased_embedding'
                           , 'nlp_random_contextual_word_substitution_bert_uncased_embedding'
                           , 'nlp_random_contextual_word_substitution_bert_cased_embedding'
                           , 'nlp_random_contextual_word_substitution_distilbert_uncased_embedding'
                           , 'nlp_random_contextual_word_substitution_distilbert_cased_embedding'
                           , 'nlp_random_contextual_word_substitution_roberta_embedding'
                           , 'nlp_random_contextual_word_substitution_distilroberta_base_embedding'
                           #, 'nlp_random_contextual_word_substitution_xlnet_embedding'
                           , 'nlp_random_contextual_word_substitution_bart_base_embedding'
                           , 'nlp_random_contextual_word_substitution_squeezebert_uncased_embedding'

                           ]
'''

supported_augment_labels = [
                           , 'nlp_random_crop'
                           , 'nlp_back_translation_aug'
    
]
'''

"\n\nsupported_augment_labels = [\n                           , 'nlp_random_crop'\n                           , 'nlp_back_translation_aug'\n    \n]\n"

In [7]:
for aug in char_augment_labels:
    
    predict_augmented_labels(aug
                            , num_samples = 4
                            , aug_char_min = 1
                            , aug_char_max = 1
                            , aug_char_p = 1
                            , aug_word_min = 1
                            , aug_word_max = None
                            , aug_word_p = 0.1
                            , min_char = 4)

num_samples: 4
char_min: 1
word_p: 0.1
char_min: 1
word_p: 0.1
nlp_random_char_substitution
initialize model
groupDRO


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertClassifier: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN t

initialization complete
generating predictions


  0%|          | 0/2091 [00:00<?, ?it/s]

writing predictions
num_samples: 4
char_min: 1
word_p: 0.1
char_min: 1
word_p: 0.1
nlp_random_char_swap
initialize model
groupDRO


...

writing predictions
num_samples: 4
char_min: 1
word_p: 0.1
char_min: 1
word_p: 0.1
nlp_random_char_deletion
initialize model
groupDRO


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertClassifier: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN t

initialization complete
generating predictions


  0%|          | 0/2091 [00:00<?, ?it/s]

writing predictions


In [7]:
for word_aug in word_augment_labels:
    
    predict_augmented_labels(word_aug
                            , num_samples = 4
                            , aug_word_min = 1
                            , aug_word_max = 1
                            , aug_word_p = 1
                            , min_char = 4)


num_samples: 4
char_min: 1
word_p: 1
char_min: 1
word_p: 1
nlp_spelling_substitution
initialize model
groupDRO


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertClassifier: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN t

initialization complete
generating predictions


  0%|          | 0/2091 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 58.00 MiB (GPU 0; 11.91 GiB total capacity; 7.23 GiB already allocated; 12.94 MiB free; 7.34 GiB reserved in total by PyTorch)

In [17]:
char_augmentations = ['nlp_ocr'
                       , 'nlp_keyboard'
                       , 'nlp_random_char_insert'
                       , 'nlp_random_char_substitution'
                       , 'nlp_random_char_swap'
                       , 'nlp_random_char_deletion']

for char_aug in char_augmentations:
    print(char_aug)
    predict_augmented_labels(char_aug
                            , num_samples = 1
                            , aug_char_min = 1
                            , aug_char_max = 1
                            , aug_char_p = 1
                            , aug_word_min = 1
                            , aug_word_max = None
                            , aug_word_p = 0.1
                            , min_char = 4)

nlp_ocr
num_samples: 1
char_min: 1
word_p: 0.1
char_min: 1
word_p: 0.1
nlp_ocr
initialize model


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertClassifier: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN t

initialization complete
generating predictions


  0%|          | 0/2091 [00:00<?, ?it/s]

writing predictions
nlp_keyboard
num_samples: 1
char_min: 1
word_p: 0.1
char_min: 1
word_p: 0.1
nlp_keyboard
initialize model


...

writing predictions
nlp_random_char_deletion
num_samples: 1
char_min: 1
word_p: 0.1
char_min: 1
word_p: 0.1
nlp_random_char_deletion
initialize model


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertClassifier: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN t

initialization complete
generating predictions


  0%|          | 0/2091 [00:00<?, ?it/s]

writing predictions


In [21]:
predict_augmented_labels('nlp_random_contextual_word_insertion_bert_uncased_embedding'
                        , num_samples = 1
                            , aug_word_min = 1
                            , aug_word_max = 1
                            , aug_word_p = 1)

num_samples: 1
char_min: 1
word_p: 1
char_min: 1
word_p: 1
nlp_random_contextual_word_insertion_bert_uncased_embedding
initialize model


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertClassifier: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN t

initialization complete
generating predictions


  0%|          | 0/2091 [00:00<?, ?it/s]

writing predictions


(array([     0,     10,     11, ..., 447989, 447993, 447997]),
 <wilds.datasets.civilcomments_dataset.CivilCommentsDataset at 0x7f51a1a982e8>)